In [1]:
import numpy as np
import csv
import matplotlib.pyplot as plt
import astropy.constants as cte
import astropy.units as un
from astropy.cosmology import Planck15 as cosmo
import emcee
import scipy
from scipy import optimize as op
import corner
import os
os.environ["OMP_NUM_THREADS"] = "8"
from multiprocessing import Pool
import time
from datetime import datetime 

from scipy.optimize import minimize

In [2]:
c=cte.c
c_um=c.to("um/s")
c_mm=c.to("mm/s")

c = c.value # m/s

Msun=cte.M_sun
Msun = Msun.value # kg

h = cte.h.value

K=cte.k_B.value

In [3]:
file = open("/Users/mariajesusfloresmoraga/Desktop/DSFG/book2.csv")
csvreader = csv.reader(file)
header = []
header = next(csvreader)

rows = []
for row in csvreader:
        rows.append(row)

ID= []

z=[]

S3=[]
errS3=[]

S2=[]
errS2=[]

S1=[]
errS1=[]

S870=[]
errS870=[]

S500=[]
errS500=[]

S350=[]
errS350=[]

S250=[]
errS250=[]

S160=[]
errS160=[]

S100=[]
errS100=[]

for j in rows:   
    
    ID.append(j[0])
    z.append(float(j[1]))
    S3.append(float(j[2]))
    errS3.append(float(j[3]))
    S2.append(float(j[4]))
    errS2.append(float(j[5]))
    S1.append(float(j[6]))
    errS1.append(float(j[7]))
    S870.append(float(j[8]))
    errS870.append(float(j[9]))
    S500.append(float(j[10]))
    errS500.append(float(j[11]))
    S350.append(float(j[12]))
    errS350.append(float(j[13]))
    S250.append(float(j[14]))
    errS250.append(float(j[15]))
    S160.append(float(j[16]))
    errS160.append(float(j[17]))
    S100.append(float(j[18]))
    errS100.append(float(j[19]))
   

In [4]:
v__=np.array([500,350,250,160,100])*un.um
v_=(c_um/v__).to("Hz")
v=v_.value
l__=np.array([95,150,220])*un.GHz
l_=l__.to("Hz")
l=l_.value
vf=np.concatenate((l,v))
print(vf)

[9.50000000e+10 1.50000000e+11 2.20000000e+11 5.99584916e+11
 8.56549880e+11 1.19916983e+12 1.87370286e+12 2.99792458e+12]


In [12]:
vf_80=[]  #lista con 80 arrays de vf
for x in range(len(rows)):
    vf_80.append(vf)

In [6]:
frecsT_l=[]
errfT_l=[]
for m in range (len(rows)):
    
    frec=[S3[m],S2[m],S1[m],S500[m],S350[m],S250[m],S160[m],S100[m]]
    errf=[errS3[m],errS2[m],errS1[m],errS500[m],errS350[m],errS250[m],errS160[m],errS100[m]]
    frecsT_l.append(frec)
    errfT_l.append(errf)
    
datos=[]  #todas estas listas son iguales. Definí varias, para después evitar borrar datos donde no queremos
datos_2=[]
datos_3=[]
datos_4=[]

for x,y in zip(frecsT_l,errfT_l):
    datos_=[]
    datos_.append(x)
    datos_.append(y)
    datos.append(datos_) #así, la lista datos está formada por muchos pares de listas. En cada par está flux y err_flux
    datos_2.append(datos_)
    datos_3.append(datos_)
    datos_4.append(datos_)

In [7]:
for i in range(len(datos)):  #ejemplo de como queda la lista
    print("Para {}; flujos:{}, errores:{}".format(ID[i],datos[i][0],datos[i][1]))

Para SPT0002-52; flujos:[0.47, 3.1, 10.7, 202.0, 283.5, 332.9, 234.3, 93.9], errores:[0.03, 0.9, 5.8, 10.0, 8.9, 10.1, 21.3, 5.4]
Para SPT0020-51; flujos:[1.25, 6.6, 19.9, 144.1, 120.9, 71.5, nan, nan], errores:[0.05, 1.4, 4.6, 8.6, 7.4, 7.5, 8.7, 2.7]
Para SPT0027-50; flujos:[1.41, 8.6, 35.6, 316.1, 325.7, 232.9, 85.6, 15.8], errores:[0.05, 1.4, 5.1, 8.1, 7.2, 6.8, 13.0, 2.3]
Para SPT0054-41; flujos:[1.8, 10.4, 32.5, 174.2, 125.9, 79.0, nan, nan], errores:[0.06, 1.5, 5.3, 10.3, 8.9, 9.3, nan, nan]
Para SPT0103-45; flujos:[1.46, 8.2, 30.6, 231.6, 213.3, 133.2, nan, nan], errores:[0.23, 1.5, 5.2, 7.6, 6.8, 10.8, 15.7, 4.4]
Para SPT0106-64; flujos:[2.15, 12.7, 42.6, 237.3, 255.7, 151.9, 64.1, nan], errores:[0.07, 1.3, 4.9, 8.7, 10.1, 8.0, 13.5, 4.1]
Para SPT0109-47; flujos:[1.11, 6.9, 15.6, 213.9, 219.4, 166.4, 57.3, nan], errores:[0.04, 1.5, 5.0, 8.4, 9.1, 8.8, 9.5, 2.8]
Para SPT0112-55; flujos:[0.18, 4.3, 17.3, 37.0, 38.0, nan, nan, nan], errores:[0.03, 1.1, 4.7, 7.3, 6.3, 6.3, nan, na

In [8]:
In=[]
for i in range(len(datos)):
    nf=[]
    ne=[]
    vT=[]
    indices=[]
    for m,n in zip(datos[i][0],datos[i][1]):
        if np.isnan(m)==True or np.isnan(n)==True:
            if datos[i][0].index(m) == datos[i][1].index(n):
                indice=datos[i][0].index(m)
                #print("indice:",indice)
                indices.append(indice)
            else:
                if datos[i][0].index(m) - datos[i][1].index(n) > 0:
                    indice=datos[i][0].index(m)
                    #print("indice:",indice)
                    indices.append(indice)
                else:
                    indice=datos[i][0].index(m)
                    #print("indice:",indice)
                    indices.append(indice)
    In.append(indices)
    
lista_i=np.linspace(0,len(datos)-1,len(datos))


for i,indice in zip(lista_i,In):
    print("indices:{},frecs:{},errs:{}".format(indice,datos_2[int(i)][0],datos_2[int(i)][1]))
    
    

                    
      

indices:[],frecs:[0.47, 3.1, 10.7, 202.0, 283.5, 332.9, 234.3, 93.9],errs:[0.03, 0.9, 5.8, 10.0, 8.9, 10.1, 21.3, 5.4]
indices:[6, 7],frecs:[1.25, 6.6, 19.9, 144.1, 120.9, 71.5, nan, nan],errs:[0.05, 1.4, 4.6, 8.6, 7.4, 7.5, 8.7, 2.7]
indices:[],frecs:[1.41, 8.6, 35.6, 316.1, 325.7, 232.9, 85.6, 15.8],errs:[0.05, 1.4, 5.1, 8.1, 7.2, 6.8, 13.0, 2.3]
indices:[6, 7],frecs:[1.8, 10.4, 32.5, 174.2, 125.9, 79.0, nan, nan],errs:[0.06, 1.5, 5.3, 10.3, 8.9, 9.3, nan, nan]
indices:[6, 7],frecs:[1.46, 8.2, 30.6, 231.6, 213.3, 133.2, nan, nan],errs:[0.23, 1.5, 5.2, 7.6, 6.8, 10.8, 15.7, 4.4]
indices:[7],frecs:[2.15, 12.7, 42.6, 237.3, 255.7, 151.9, 64.1, nan],errs:[0.07, 1.3, 4.9, 8.7, 10.1, 8.0, 13.5, 4.1]
indices:[7],frecs:[1.11, 6.9, 15.6, 213.9, 219.4, 166.4, 57.3, nan],errs:[0.04, 1.5, 5.0, 8.4, 9.1, 8.8, 9.5, 2.8]
indices:[5, 6, 7],frecs:[0.18, 4.3, 17.3, 37.0, 38.0, nan, nan, nan],errs:[0.03, 1.1, 4.7, 7.3, 6.3, 6.3, nan, nan]
indices:[6, 7],frecs:[1.28, 8.9, 22.3, 89.1, 56.2, 24.9, nan, na

In [11]:
new_f=[]
new_e=[]
new_vf=[]


for i,indice in zip(lista_i,In):
    
    if len(indice)==0:
        new_f.append(datos_2[int(i)][0])
        new_e.append(datos_2[int(i)][1])
        new_vf.append(vf_80[int(i)])
        print("8:",len(datos_2[int(i)][0]),len(datos_2[int(i)][1]),len(vf_80[int(i)])) #todo esto debería tener 8 de largo
              
        
    if len(indice)==1:
        
        x1=np.delete(datos_2[int(i)][0],indice[0])
        x2=np.delete(datos_2[int(i)][1],indice[0])
        x3=np.delete(vf_80[int(i)],indice[0])
        
        print("7:",len(x1),len(x2),len(x3)) #esto debería tener 7 de largo
        
        new_f.append(x1)
        new_e.append(x2)
        new_vf.append(x3)
        
    if len(indice)==2:
        
        ind_menor=np.sort(indice)[0]
        ind_mayor=np.sort(indice)[1]
        
        
        del (datos_3[int(i)][0])[ind_menor:ind_mayor]
        del (datos_3[int(i)][1])[ind_menor:ind_mayor]
        del (list(vf_80[int(i)]))[ind_menor:ind_mayor]
        
        print("6:",len(datos_3[int(i)][0]),len(datos_3[int(i)][1]),len(list(vf_80[int(i)])))
        #esto debería tener 6 de largo
        
        new_f.append(datos_2[int(i)][0])
        new_e.append(datos_2[int(i)][1])
        new_vf.append(vf_80[int(i)])
        
    if len(indice)==3:
        print("3")
        ind_menor=np.sort(indice)[0]
        ind_med=np.sort(indice)[1]
        ind_mayor=np.sort(indice)[2]
        
        del (datos_4[int(i)][0])[ind_menor:ind_med:ind_mayor]
        del (datos_4[int(i)][1])[ind_menor:ind_med:ind_mayor]
        del (list(vf_80[int(i)]))[ind_menor:ind_med:ind_mayor]
        
        print("5:",len(datos_4[int(i)][0]),len(datos_4[int(i)][1]),len(list(vf_80[int(i)])))
        #esto debería tener 5 de largo
        
        
        new_f.append(datos_2[int(i)][0])
        new_e.append(datos_2[int(i)][1])
        new_vf.append(vf_80[int(i)])
        


8: 8 8 8
6: 6 6 8
8: 8 8 8
6: 6 6 8
6: 6 6 8
7: 7 7 7
7: 7 7 7
3
5: 7 7 8
6: 6 6 8
8: 8 8 8
8: 8 8 8
6: 6 6 8
6: 6 6 8
6: 6 6 8
6: 6 6 8
8: 8 8 8
6: 6 6 8
6: 6 6 8
8: 8 8 8
6: 6 6 8
6: 6 6 8
7: 7 7 7
7: 7 7 7
8: 8 8 8
7: 7 7 7
6: 6 6 8
6: 6 6 8
6: 6 6 8
7: 7 7 7
6: 6 6 8
6: 6 6 8
6: 6 6 8
6: 6 6 8
6: 6 6 8
6: 6 6 8
6: 6 6 8
8: 8 8 8
8: 8 8 8
6: 6 6 8
6: 6 6 8
6: 6 6 8
7: 7 7 7
7: 7 7 7
6: 6 6 8
8: 8 8 8
3
5: 7 7 8
8: 8 8 8
6: 6 6 8
6: 6 6 8
8: 8 8 8
8: 8 8 8
6: 6 6 8
8: 8 8 8
6: 6 6 8
8: 8 8 8
6: 6 6 8
6: 6 6 8
6: 6 6 8
6: 6 6 8
8: 8 8 8
6: 6 6 8
8: 8 8 8
6: 6 6 8
7: 7 7 7
7: 7 7 7
6: 6 6 8
6: 6 6 8
8: 8 8 8
7: 7 7 7
6: 6 6 8
6: 6 6 8
7: 7 7 7
6: 6 6 8
6: 6 6 8
6: 6 6 8
6: 6 6 8
6: 6 6 8
6: 6 6 8
6: 6 6 8
8: 8 8 8
6: 6 6 8
